In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install flask-restx

     |████████████████████████████████| 5.3MB 30.9MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
# git_repo_url = 'https://github.com/dinatih/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949

  print("install new CMake becaue of CUDA10")
  if not exists('cmake-3.13.0-Linux-x86_64.tar.gz'):
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("CMakelist.txt's caffe fix")
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


import os
from os.path import exists, join, basename, splitext
colab_video_path = '/content/drive/My Drive/gims-nabila.mp4'
colab_openpose_video_path = colab_video_path.replace('.mp4', '') + '-openpose.mp4'

!cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 12 --video '{colab_video_path}' --display 0 --write_video_with_audio --write_video '{colab_openpose_video_path}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25


In [ ]:
from flask import *
from flask_ngrok import run_with_ngrok
from flask_restx import Namespace, Resource,Api
from IPython.display import Image
from werkzeug.utils import secure_filename
import os
from os.path import exists, join, basename, splitext
from io import StringIO
import base64
#from PIL import Image
from io import BytesIO
import numpy as np
from torchvision import transforms
import math,time
import json 
import requests
from __future__ import print_function

import cv2
from google.colab.patches import cv2_imshow
!pip install opencv-python # 굳이 실행 안시켜도 됨(버전 안맞으면 돌리기)


if not os.path.exists("/content/img"):
  os.makedirs("/content/img")
if not os.path.exists("/content/preimg"):
  os.makedirs("/content/preimg")



flask_app = Flask(__name__)
flask_app.config["IMAGE_UPLOADS"] = "/content/img/"
flask_app.config["PRE_IMAGE_UPLOADS"] = "/content/preimg/"
app=Api(flask_app)

predict_ns = app.namespace('Predict', description='this is predict file for flask')

swagger_parser = predict_ns.parser()
prejsonfilepath=0

@predict_ns.route('/prepare')
class Predict(Resource):
    """ POST로 받은 이미지를 분류하는 API """

    @predict_ns.doc(responses={
        200: 'OK',
        204: 'Fail to find a face',
        400: 'Bad Request',
        500: 'Internal Server Error'
    })
    @predict_ns.expect(swagger_parser)
    def post(self):

      
      if "file" not in request.files :
        print('no image')
        return 'no image' #에서 500 에러 발생(아직 이미지 안보내줬으므로)
  
      print('////////////////////////////')
      f=request.files['file']
      input_image = f.stream.read()
      f.seek(0)
      
      
      print(input_image)        #결과 : b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x0
      print(request.files)      #결과 : ImmutableMultiDict([('image', <FileStorage: '428185a27687528890204b20d2bd55cb.jpg' ('image/jpeg')>)])
      display(Image(input_image))

      print(f)                              #결과 : <FileStorage: '42d57cdeab87ce9ea805eb7c7cbb852d.jpg' ('image/jpeg')>
      directory="/content/preimg/"+f.filename
      if not os.path.exists(directory):
            os.makedirs(directory)

      f.save(os.path.join(directory+"/", f.filename))

      print(secure_filename(f.filename))    #결과 : 41889c3ccf041e3da2a0378de8125d75.jpg
      
    
    
     # colab_openpose_video_path = colab_video_path.replace('.jpg', '') + '-openpose.jpg'
      colab_directory="/content/prejson/"+f.filename
      if not os.path.exists(colab_directory):
            os.makedirs(colab_directory)
      colab_json_path=colab_directory  #jsonifle 저장되는 경로


      
      folder_path = directory+"/"  #폴더명 내걸로 변경
      files = os.listdir(folder_path)
      print(files)
      files.reverse()
      for filename in files:
        print(filename)
        colab_video_path = folder_path + filename #input 파일
        print(colab_video_path)
        colab_openpose_video_path = '/content/drive/MyDrive/Colab Notebooks/deskdata//open_image/'+filename
        print(colab_openpose_video_path+'\n\n')
        #if not exists(colab_openpose_video_path):
        !cd openpose && ./build/examples/openpose/openpose.bin --video  '{colab_video_path}'  --render_pose 0 --display 0 --number_people_max=2 --write_json '{colab_json_path}'
        #!cd openpose && ./build/examples/openpose/openpose.bin --hand --face --number_people_max 12 --video '{colab_video_path}' --display 0  --write_video '{colab_openpose_video_path}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25

      jsonfloder_path = colab_json_path+'/'
      files = os.listdir(jsonfloder_path)
      print(files)
      files.reverse()
      for filename in files:
        result = 0
        flag= 0; #flag가 0이면 올바른 자세로 인식 후 저장

        global prejsonfilepath
        
        #json_file = jsonfloder_path + filename #input 파일

        prejsonfilepath = jsonfloder_path + filename
        print(prejsonfilepath)


        

        
        with open(jsonfloder_path + filename) as json_file:
          keydata = json.load(json_file) 
          jsonArray= keydata.get("people")

          
          li = []
          for list in jsonArray:
            li.append(list.get("pose_keypoints_2d"))

    
        #li 라는 2차원 배열에 저장된 위치값들  
        #3k=x위치, 3k+1=y위치 3k+2=신뢰구간




        #아예 필요한 모든 관절이 모니터상에 없는 경우 : 가장 첫번쨰에 떠야할 알람!
        #어떤 자세이든 간에 일단 꼭 검출이 되어야 하는 data는 0,1 2,5,15,16,17,18 이 포인트들은 무조건 검출이 되어야함
        # 하나라도 0 값이 나온다면 모니터에 어깨와 얼굴이 다 나오도록 자세를 조정해주세요! 라는 알림을 보낸다. 

          all=0
          all=(li[0][0]*li[0][3]*li[0][6]*li[0][15]*li[0][45]*li[0][48]*li[0][51]*li[0][54])

          if all==0 :
            print("모니터에 어깨와 얼굴이 다 나오도록 자세를 조정해주세요!")
            #result[0] = 1
            flag=1 #flag가 1이면 웹캠화면에 자세 다 안나온 경우 이므로 다시찍게 하기
            response={"text" : result, "flag":flag}
            return response,200
          else:
            print ("화면에 자세가 보입니다")
            
          #이게 if문을 해결못하면 계속 false, 이걸 해결하면 아래의 조건식으로 넘어가게!


          #어깨 불균형 : point 2&5 의 기울기 말고 그냥 y 좌표로 하면 안될까? (오차범위 설정)
          sh_y=0
          sh_y= li[0][7]-li[0][16]

          if  (-10 >=sh_y )  :
            print("지금 오른쪽 어깨가 왼쪽보다 올라갔군요!")
            result="오른쪽 어깨"
            flag=2 #자세가 안좋은 경우 flag를 2로 하고 result에 어디가 안좋은지 그 내용 담음
          elif (sh_y >=10):
            print("지금 왼쪽어깨가 오른쪽보다 올라갔군요!")
            result="왼쪽 어깨"
            flag=2 #자세가 안좋은 경우 flag를 2로 하고 result에 어디가 안좋은지 그 내용 담음
          else :
            print("어깨 자세가 좋아요!")
             
    
    

          #비스듬한 고개: 0-1  x축 위치 차이가 잘못된 자세의 경우 100이상 차이가 난다. 바른 자세는 거의 차이가 안남 (10 정도?)

          neck_x = li[0][0]-li[0][3]


          if  (-40 >=neck_x)  :
            print("지금 고개가 오른쪽으로 기울었군요!")
            result="오른쪽"
            flag=3
          elif (neck_x>40):
            print("지금 고개가 왼쪽으로 기울었군요!")
            result="왼쪽"
            flag=3
          else :
              print("고개 자세가 바르군요")
              
    



      response={"text" : result, "flag":flag}
      return response,200

    


    def get(self):
       return  '....'

@predict_ns.route('/predict')
class Predict(Resource):
    """ POST로 받은 이미지를 분류하는 API """

    @predict_ns.doc(responses={
        200: 'OK',
        204: 'Fail to find a face',
        400: 'Bad Request',
        500: 'Internal Server Error'
    })
    @predict_ns.expect(swagger_parser)
    def post(self):

      
      if "file" not in request.files :
        print('no image')
        return 'no image' #에서 500 에러 발생(아직 이미지 안보내줬으므로)
    
      print('////////////////////////////')
      f=request.files['file']
      input_image = f.stream.read()
      f.seek(0)
      
      
      print(input_image)        #결과 : b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x0
      print(request.files)      #결과 : ImmutableMultiDict([('image', <FileStorage: '428185a27687528890204b20d2bd55cb.jpg' ('image/jpeg')>)])
      display(Image(input_image))
      inputfilename=f.filename
      print(f)                              #결과 : <FileStorage: '42d57cdeab87ce9ea805eb7c7cbb852d.jpg' ('image/jpeg')>
      directory="/content/img/"+f.filename  #결과 :/content/img/hayoung1214.jpg
      if not os.path.exists(directory):
        os.makedirs(directory)

      f.save(os.path.join(directory+"/", f.filename))

      print(secure_filename(f.filename))    #결과 : 41889c3ccf041e3da2a0378de8125d75.jpg
      
      
      # colab_openpose_video_path = colab_video_path.replace('.jpg', '') + '-openpose.jpg'
      colab_directory="/content/json/"+f.filename
      if not os.path.exists(colab_directory):
        os.makedirs(colab_directory)
      colab_json_path=colab_directory  #jsonifle 저장되는 경로 #결과: /content/json/hayoung1214.jpg

      #Yolo 손 알고리즘 시작 부분
      folder_path = directory+"/"  #폴더명 내걸로 변경 #결과 :/content/img/hayoung1214.jpg/
      files = os.listdir(folder_path)
      print("files : ")
      print(files)
      files.reverse()
      for filename in files:
        result = 0
        flag= 0; #flag가 0이면 올바른 자세로 인식 후 저장

        print(",,,,,,,,,,,,,")
        print(filename)
        colab_hand_path = folder_path + filename #input 파일 #결과 :/content/img/hayoung1214.jpg/hayoung.jpg
        print(colab_hand_path)
        
        # Yolo 로드
        net = cv2.dnn.readNet("/content/drive/MyDrive/use/yolov3_custom_train_2900.weights", "/content/drive/MyDrive/use/yolov3_custom_train.cfg")#이 셋 폴더를 final_use_yolo랑 같은 디렉토리(구글 드라이브)에 저장해놓기
        classes = []                                                                                #같은 디렉토리이면 경로명 수정 안해도됨
        with open("/content/drive/MyDrive/use/yolo.names", "r") as f:
            classes = [line.strip() for line in f.readlines()]
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        # 이미지 가져오기 #웹캠 이미지 
        img = cv2.imread(colab_hand_path) #이자리에 괄호 없이 이미지 파일(웹캠) 이름 바로 넣기
        img = cv2.resize(img, None, fx=0.4, fy=0.4)
        height, width, channels = img.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(img, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)


        # 정보를 화면에 표시
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.7: #70프로 이상이면 바운딩 박스 그리는 부분( 우리는 필요없어서 나중에 지워도 됨 그럼 속도 빨라질듯)
                    # Object detected
                    print(confidence) #confirence 가 숫자값 이게 중요 이거 대신에 reponse로 손(턱) 알림 보내기
                    print("손이 검출되었습니다.")
                    result="손"
                    flag=5
                    response={"text" : result, "flag":flag}
                    return response,200
        print("손이 검출되지 않았습니다.")          
          
      
        colab_video_path = folder_path + filename #input 파일
        print(colab_video_path)
        colab_openpose_video_path = '/content/drive/MyDrive/Colab Notebooks/deskdata//open_image/'+filename
        print(colab_openpose_video_path+'\n\n')
        #if not exists(colab_openpose_video_path):
        !cd openpose && ./build/examples/openpose/openpose.bin --video  '{colab_video_path}'  --render_pose 0 --display 0 --number_people_max=2 --write_json '{colab_json_path}'
        #!cd openpose && ./build/examples/openpose/openpose.bin --hand --face --number_people_max 12 --video '{colab_video_path}' --display 0  --write_video '{colab_openpose_video_path}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25


      

      jsonfloder_path = colab_json_path+'/'
      files = os.listdir(jsonfloder_path)
      print(files)
      files.reverse()
      for filename in files:
        result = 0
        flag= 0; #flag가 0이면 올바른 자세로 인식 후 저장

        
        print(filename)
        #json_file = jsonfloder_path + filename #input 파일
        with open(jsonfloder_path + filename) as json_file:
          keydata = json.load(json_file) 
          jsonArray= keydata.get("people")

          global prejsonfilepath
          with open(prejsonfilepath) as json_file:
            prekeydata = json.load(json_file) 
            prejsonArray= prekeydata.get("people")

            preli=[]
            for list in prejsonArray:
              preli.append(list.get("pose_keypoints_2d"))

            
          li = []
          for list in jsonArray:
            li.append(list.get("pose_keypoints_2d"))

    
        #li 라는 2차원 배열에 저장된 위치값들  
        #3k=x위치, 3k+1=y위치 3k+2=신뢰구간




        #아예 필요한 모든 관절이 모니터상에 없는 경우 : 가장 첫번쨰에 떠야할 알람!
        #어떤 자세이든 간에 일단 꼭 검출이 되어야 하는 data는 0,1 2,5,15,16,17,18 이 포인트들은 무조건 검출이 되어야함
        # 하나라도 0 값이 나온다면 모니터에 어깨와 얼굴이 다 나오도록 자세를 조정해주세요! 라는 알림을 보낸다. 

          all=0
          all=(li[0][0]*li[0][3]*li[0][6]*li[0][15]*li[0][45]*li[0][48]*li[0][51]*li[0][54])

          if all==0 :
            print("모니터에 어깨와 얼굴이 다 나오도록 자세를 조정해주세요!")
            flag=1
            response={"text" : result, "flag":flag}
            return response,200
          else:
            print ("화면에 자세가 보입니다")
            
          #이게 if문을 해결못하면 계속 false, 이걸 해결하면 아래의 조건식으로 넘어가게!


          #어깨 불균형 : point 2&5 의 기울기 말고 그냥 y 좌표로 하면 안될까? (오차범위 설정)
          sh_y=0
          sh_y= li[0][7]-li[0][16]

          if  (-10 >=sh_y )  :
            print("지금 오른쪽 어깨가 왼쪽보다 올라갔군요!")
            result="오른쪽 어깨"
            flag=2 #자세가 안좋은 경우 flag를 2로 하고 result에 어디가 안좋은지 그 내용 담음
            response={"text" : result, "flag":flag}
            return response,200
          elif (sh_y >=10):
            print("지금 왼쪽어깨가 오른쪽보다 올라갔군요!")
            result="왼쪽 어깨"
            flag=2 #자세가 안좋은 경우 flag를 2로 하고 result에 어디가 안좋은지 그 내용 담음
            response={"text" : result, "flag":flag}
            return response,200
          else :
              print("어깨 자세가 좋아요!")
             
    
    

          #비스듬한 고개: 0-1  x축 위치 차이가 잘못된 자세의 경우 100이상 차이가 난다. 바른 자세는 거의 차이가 안남 (10 정도?)

          neck_x = li[0][0]-li[0][3]


          if  (-40 >=neck_x)  :
            print("지금 고개가 오른쪽으로 기울었군요!")
            result="오른쪽"
            flag=3
            response={"text" : result, "flag":flag}
            return response,200
          elif (neck_x>40):
            print("지금 고개가 왼쪽으로 기울었군요!")
            result="왼쪽"
            flag=3
            response={"text" : result, "flag":flag}
            return response,200
          else :
              print("고개 자세가 바르군요")
              
    

          #flask 코드 내부에서 자세가 좋다는 알람이 두 번 이상 간다면 해당 값을 저장한다: 하영이한테 말하기



          #거북목: 0-1 #단순히 길이뿐만 아니라 각 관절들의 비율을 찾아야할듯 
          preneck_x = preli[0][0]-preli[0][3]
          preneck_y= preli[0][1]-preli[0][4]
          stan_len=math.sqrt((preneck_x  ** 2) + (preneck_y ** 2))
          print(stan_len)
          print(preli[0][4])
          print(preli[0][1])
          #stan_len=150 #초기 저장했던 길이 (150 대신에 초기 셋팅 길이 넣기(기준길이))
          #이때의 stan_len 값은 true 라고 확정된 상태의 목의 길이 값(초기 사용 시 사용자에게 해당 값을 고정할 시간을 마련한다.)
      
          #neck_x 는 바로 위에 있음

          neck_y= li[0][1]-li[0][4]

          new_turtle = math.sqrt((neck_x  ** 2) + (neck_y ** 2)) #x변화량 y변화량 new_turtle은 현재 프레임 값(새로 구하고자하는 목 길이)
          stan_ang= 15 / 180 #stan_ang을 벗어나면 거북목임(이건 건들필요 없음)
      
          com_turtle= math.cos(stan_ang*math.pi)*stan_len #com_turtle은  거북목을 계산한다(진짜로 비교할 값)

          if new_turtle < com_turtle :
            result="거북목"
            flag=4
            response={"text" : result, "flag":flag}
            return response,200

          else :
            print("거북목 위험 자세가 아니에요")
            
            
          #턱을 괜 자세 1&7 / 1&4 (0,17,18) 가 특정 거리만큼 가까워져있는 경우.
          #이 부분 너무 어려움
    
    
    
          #눈 관련 detect 가 어디까지 가능한지 확인하기 
          #확인 결과: 눈의 이동은 바로 잘 알아채지만 동일 위치에서의 눈동자만의 이동은 잘 인식하지 못하는 것 같다. 


      response={"text" : result, "flag":flag}
      return response,200

    
      
      

    def get(self):
       return  '....'



@predict_ns.route('/main')
class Main(Resource):
  def get(self):
    return 'Hello World'



run_with_ngrok(flask_app)
flask_app.run() 

    